[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jussippjokinen/CogMod-Tutorial/blob/main/02-DeepRL/Gaze_Based_Interaction/gaze_based_interaction.ipynb)


# A cognitive model of gaze-based interaction

Some cognitive models describe behaviour and others predict it. Models that predict behaviour must be capable of generating output without that output being part of the input. In this notebook we demonstrate this property for a model of eye movements. The model is a reinforcement learner that is trained by performing hundreds of thousands of simulated eye movements in search of a target of varying size and distance. The model predicts how many eye movements people will tend to make to find a target of a given size and distance. It predicts inhibition of return. It **predicts** Fitts's Law like behaviour and it predicts that the first eye movement will usually undershoot, rather than overshoot the target.

<img src="https://raw.githubusercontent.com/howesa/CHI22-CogMod-Tutorial/main/03-Reinforcement-Learning/034_Gaze_based_Interaction/image/sub_movements.png" alt="Corati Modeling" width="400">

(source: Meyer, D. E., Abrams, R. A., Kornblum, S., Wright, C. E., & Keith Smith, J. E. (1988). Optimality in human motor performance: ideal control of rapid aimed movements. Psychological review, 95(3), 340.)

Note that what might seem the obvious strategy -- aim for where you believe the target is -- is not necessarily the optimal strategy.

### The task: Gaze-based interaction

Gaze-based interaction is a mode of interaction in which users, including users with a range of movement disabilities, are able to indicate which display item they wish to select by fixating their eyes on it. Confirmation of selection is then made with one of a number of methods, including with a key press, or by holding the fixation for an extended duration of time. The model performs this task for targets with randomly selected location and size.

<img src="https://raw.githubusercontent.com/howesa/CHI22-CogMod-Tutorial/main/03-Reinforcement-Learning/034_Gaze_based_Interaction/image/gaze_task.png" alt="Corati Modeling" width="400">

In the figure, the red lines represent saccades (eye movements). Multiple eye movements are needed to reach the target (the black circle).

### Model architecture

The model has a simple architecture that you have previously seen in the introduction. The figure is reproduced here:

<img src="https://raw.githubusercontent.com/howesa/CHI22-CogMod-Tutorial/main/03-Reinforcement-Learning/034_Gaze_based_Interaction/image/cog_arch.png" alt="Corati Modeling" width="700">

- The **control** module makes decisions about where to move the eyes with the oculomotor system. Decisions are conditioned on the current belief about the location of the target.
- The **motor** module implements decisions but it is bounded by Gaussian noise, which models noise in the human motor system.
- The **environment** models the physics of the world and the task (the location of the target). Given a response from the motor system, a saccade is made to the aim point of the eyes, and a fixation is initiated.
- The **perception** module simulates the human capacity to localize a target with foveated vision. The accuracy of the location estimate generated by perception is negatively affected by the eccentricity of the target from the current fixation location.
- The **Memory** module stores a representation of the current state. Over the course of an episode a sequence of location estimates will be made. Humans are known to integrate these estimates into a single integrated representation of the location. People are known to do this optimally using a process that can be modelled with Bayesian state estimation. The state estimation constitutes a belief about the location of the target.
- The **Utility** module calculates a reward signal given the current belief about the enviornment. The reward signal is used to train the controller.

### Prerequisites

Before proceeding with this notebook you should firrst review the notebooks on foveated vision and on Bayesian integration. These explain how the perception and memory modules work.



### Machine learning

In order to learn how to perform the task, the model uses implementations of reinforcement learning algorithms in PyTorch known as stable-baselines3.

In [22]:
# Install stable_baselines3 and the gymnasium environment
# This is a well known machine learning library that provides a suite of reinforcement learning methods.
# Only needs to be run once

!pip install --pre -U stable_baselines3

import gymnasium as gym
from gymnasium import spaces

In [23]:
# Load local modules
# gazetools is a module that contains support functions for modeling gaze-based interaction.
# the code below makes use of them but we do not need to understand how they work in this tutorial.

!wget https://raw.githubusercontent.com/jussippjokinen/CogMod-Tutorial/main/02-DeepRL/Gaze_Based_Interaction/gazetools.py

import gazetools

--2024-11-24 23:37:27--  https://raw.githubusercontent.com/jussippjokinen/CogMod-Tutorial/main/02-DeepRL/Gaze_Based_Interaction/gazetools.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5204 (5.1K) [text/plain]
Saving to: ‘gazetools.py.1’

gazetools.py.1      100%[===================>]   5.08K  --.-KB/s    in 0s      

2024-11-24 23:37:27 (53.6 MB/s) - ‘gazetools.py.1’ saved [5204/5204]



In [24]:
# load required standard modules and configure matplotlib

import numpy as np
import math
import matplotlib.pyplot as plt
import sys

import matplotlib as mpl
%matplotlib inline
mpl.style.use('ggplot')

### Implementation of the Cognitive Architecture as a Python Class

The first step to formalise the model architecture presented in the above figure. We do this by specifying a class of cognitive theories and will later define an instance of this class.

The class has only a single method, which defines a cycle through the processes defined in the figure.

In [25]:
class CognitivePOMDP():

    def __init__(self):
        self.internal_state = {}

    def step(self, ext, action):
        ''' Define the cognitive architecture.'''
        self._update_state_with_action(action)
        response = self._get_response()
        external_state, done = ext.external_env(response)
        stimulus, stimulus_std = self._get_stimulus(ext.external_state)
        self._update_state_with_stimulus(stimulus, stimulus_std)
        obs = self._get_obs()
        reward = self._get_reward()
        return obs, reward, done

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### A theory of gaze-based interaction

Each of the entities in the cognitive architecture must be defined so as to make explicit our theory of gaze-based interaction. The theory makes the following assumptions:

- The perception of the target location is corrupted by Gaussian noise in human vision.
- The standard deviation of noise increases linearly with eccentricity from the fovea.
- Sequences of stimuli are noisily perceived and optimally integrated.
- Intended eye movements (oculomotor actions) are corrupted by signal dependent Gaussian noise to generate responses.

These assumptions are further described in Chen et al. (2021).

In [26]:
class GazeTheory(CognitivePOMDP):

    def __init__(self):
        ''' Initialise the theoretically motivated parameters.'''
        # weight eye movement noise with distance of saccade
        self.oculamotor_noise_weight = 0.01
        # weight noise with eccentricity
        self.stimulus_noise_weight = 0.09
        # step_cost for the reward function
        self.step_cost = -1
        # super.__init__()

    def reset_internal_env(self, external_state):
        ''' The internal state includes the fixation location, the latest estimate of
        the target location and the target uncertainty. Assumes that there is no
        uncertainty in the fixation location.
        Assumes that width is known. All numbers are on scale -1 to 1.
        The target_std represents the strength of the prior.'''
        self.internal_state = {'fixation': np.array([-1,-1]),
                               'target': np.array([0,0]),
                               'target_std': 0.1,
                               'width': external_state['width'],
                               'action': np.array([-1,-1])}
        return self._get_obs()

    def _update_state_with_action(self, action):
        self.internal_state['action'] = action

    def _get_response(self):
        ''' Take an action and add noise.'''
        # !!!! should take internal_state as parameter
        move_distance = gazetools.get_distance( self.internal_state['fixation'],
                                     self.internal_state['action'] )

        ocularmotor_noise = np.random.normal(0, self.oculamotor_noise_weight * move_distance,
                                        self.internal_state['action'].shape)
        # response is action plus noise
        response = self.internal_state['action'] + ocularmotor_noise

        # update the ocularmotor state (internal)
        self.internal_state['fixation'] = response

        # make an adjustment if response is out of range.
        response = np.clip(response,-1,1)
        return response

    def _get_stimulus(self, external_state):
        ''' define a psychologically plausible stimulus function in which acuity
        falls off with eccentricity.'''
        eccentricity = gazetools.get_distance( external_state['target'], external_state['fixation'] )
        stm_std = self.stimulus_noise_weight * eccentricity
        stimulus_noise = np.random.normal(0, stm_std,
                                         external_state['target'].shape)
        # stimulus is the external target location plus noise
        stm = external_state['target'] + stimulus_noise
        return stm, stm_std


    def _update_state_with_stimulus(self, stimulus, stimulus_std):
        posterior, posterior_std = self.bayes_update(stimulus,
                                                     stimulus_std,
                                                     self.internal_state['target'],
                                                     self.internal_state['target_std'])
        self.internal_state['target'] = posterior
        self.internal_state['target_std'] = posterior_std

    def bayes_update(self, stimulus, stimulus_std, belief, belief_std):
        ''' A Bayes optimal function that integrates multiple stimului.
        The belief is the prior.'''
        z1, sigma1 = stimulus, stimulus_std
        z2, sigma2 = belief, belief_std
        w1 = sigma2**2 / (sigma1**2 + sigma2**2)
        w2 = sigma1**2 / (sigma1**2 + sigma2**2)
        posterior = w1*z1 + w2*z2
        posterior_std = np.sqrt( (sigma1**2 * sigma2**2)/(sigma1**2 + sigma2**2) )
        return posterior, posterior_std

    def _get_obs(self):
        # the Bayesian posterior has already been calculated so just return it.
        # also return the target_std so that the controller knows the uncertainty
        # of the observation.
        #return self.internal_state['target']
        return np.array([self.internal_state['target'][0],
                        self.internal_state['target'][1],
                        self.internal_state['target_std']])

    def _get_reward(self):
        distance = gazetools.get_distance(self.internal_state['fixation'],
                                self.internal_state['target'])

        if distance < self.internal_state['width'] / 2:
            reward = 0
        else:
            reward = -distance # a much better model of the psychological reward function is possible.

        return reward

### Task environment

In order to test the theory we need to define the task environment.

The task environment allows the theory to make predictions for a particular task. The theory makes predictions for many more tasks. For example, adaptation to mixed target widths and distances.

In [27]:
class GazeTask():

    def __init__(self,target_width = 0.15):
        self.target_width = target_width
        self.target_loc_std = 0.3

    def reset_external_env(self):
        ''' The external_state includes the fixation and target location.
        Choose a new target location and reset fixation to the first fixation location.'''

        def _get_new_target():
            x_target =np.clip(np.random.normal(0, self.target_loc_std),-1,1)
            y_target =np.clip(np.random.normal(0, self.target_loc_std),-1,1)
            return np.array( [x_target, y_target] )

        fx = np.array([-1,-1])
        tg = _get_new_target()
        self.external_state = {'fixation':fx, 'target':tg, 'width':self.target_width }

    def external_env(self, action):
        self.external_state['fixation'] = action

        # determine when the goal has been achieved.
        distance = gazetools.get_distance(self.external_state['fixation'],
                                self.external_state['target'])
        if distance < self.external_state['width']/2 :
            done = True
        else:
            done = False

        return self.external_state, done


### Gym (Gymnasium)

In order to find an optimal policy we use the theory and external environment to define a machine learning problem, here, making use of the framework defined by one specific library called gym.

For further information see: https://gymnasium.farama.org/

gym.Env is a class provided by this library. Note that all of the modules of the cognitive architecture are part of gym.env except for the controller.

In [28]:
class GazeModel(gym.Env):

    def __init__(self, target_width = 0.15):

        def default_box(x):
            return spaces.Box(low=-1, high=1, shape=(x, ), dtype=np.float64)

        self.GT = GazeTheory()
        self.TX = GazeTask()

        # Required by gym. These define the range of each variable.
        # Each action has an x,y coordinate therefore the box size is 2.
        # Each obs has a an x,y and an uncertainty therefore the box size is 3.
        self.action_space = default_box(2)
        self.observation_space = default_box(3)

        # max_fixations per episode. Used to curtail exploration early in training.
        self.max_steps = 500

    def reset(self, seed=None, options=None):
        ''' reset the model.'''
        self.n_step = 0
        self.TX.reset_external_env()
        self.GT.reset_internal_env(self.TX.external_state)
        obs = self.GT.reset_internal_env( self.TX.external_state )
        return obs, {}

    def step(self, action):
        ''' Step through one cycle of the model.'''
        obs, reward, done = self.GT.step( self.TX, action )
        self.n_step+=1

        # give up if been looking for too long
        if self.n_step > self.max_steps:
            done = True

        info = self.get_info()
        truncated = False
        return obs, reward, done, truncated, info

    def get_info(self):
        return {'step': self.n_step,
                'target_width': self.TX.target_width,
                'target_x': self.TX.external_state['target'][0],
                'target_y': self.TX.external_state['target'][1],
                'fixate_x':self.TX.external_state['fixation'][0],
                'fixate_y':self.TX.external_state['fixation'][1] }

### Test the model

Step through the untrained model to check for simple bugs. More comprehensive tests needed.

In [29]:
model = GazeModel()

model.reset()

i=0
done = False
while not done:
    # make a step with a randomly sampled action
    obs, reward, done, truncated, info = model.step(model.action_space.sample())
    i+=1

print(i)

21


### Train the model

We can train the model to generate a policy for the controller.

By plotting the learning curve we can see whether the performance improves with training and whether the model approaches an optimum performance. We are interested in approximately optimal performance, so if the training curve is not approaching asymptote then we need to train with more timesteps or revise the model.

We can see that at first the model uses hundreds of fixations to find the target, this is because it has not yet learned to move the gaze in a way that is informed by the observation. As it learns to do this, it takes fewer steps to gaze at the target and its performance improves.

If our problem definition is correct then the model will get more 'human-like' the more that it is trained. In other words, training makes it a better model of interaction.

If we assume that people are computationally rational then the optimal solution to a cognitive problem predicts human behavior.

In [ ]:
timesteps = 200000

controller = gazetools.train(model, timesteps)

In [ ]:
gazetools.plot_learning_curve()

### Increase timesteps

100,000 timesteps is not enough to train this model. Try doubling the number of timesteps and train again.

### Run the model for N trials
Run the trained model, save a trace of each episode to csv file, and animate the results.

In [ ]:
data = gazetools.run_model( model, controller, 100, 'behaviour_trace.csv' )

gazetools.animate_multiple_episodes(data, n=30)

### Exercises

- Rerun the model with different parameter settings. Start by trying a different target size. What is the impact on the behaviour?
- Discuss in groups how you would use the methods presented in the notebook to build a cognitive model of another task.

In [ ]:
def add_movement_time(data):
    data["fixation_duration"] = np.random.uniform(0.1, 1, size=len(data))
    data["saccade_duration"] = np.random.uniform(0.01, 0.2, size=len(data))

    return data

In [ ]:
datas = []
for target_width in [0.1, 0.15, 0.2, 0.3, 0.4]: # Change widths here
    model = GazeModel(target_width = target_width) # You can change the parameter 'target_width'.
    data = gazetools.run_model( model, controller, 100, 'behaviour_trace.csv' )
    datas.append(data)

In [ ]:
import pandas as pd

In [ ]:
data = pd.concat(datas)
data = add_movement_time(data)

In [ ]:
data

In [ ]:
def cal_episode_time(data):
  data['total_time']= data['fixation_duration'] + data['saccade_duration']
  episode_stats = data.groupby('episode', 'target_width').agg({'total_time': 'mean', 'episode': 'size'}).rename(columns={'episode': 'num_fixations'}).reset_index()
  final_stats = episode_stats.groupby['target_width'].agg({'total_time': ['mean', 'std'], 'num_fixations': 'mean'}).round(4)
  final_stats.columns = ['total_time_mean','total_time_std','num_fixations_mean']
  final_stats = final_stats.reset_index()
  return final_stats

In [ ]:
def add_movement_time(data):
  df =data.copy()
  df["fixation_duration"] = np.random.uniform(0.1, 0.5, size=len(data))
  df['distance']=np.sqrt((df['fixate_x'] - df['target_x'])**2 + (df['fixate_y'] - df['target_y'])**2)
  base_saccade_duration = 0.02
  distance_factor = 0.05
  df["saccade_duration"] = base_saccade_duration + distance_factor * df['distance'] + np.random.normal(0,0.01,size=len(df))

  df['saccade_duration'] = df['saccade_duration'].clip(0.02,0.1)
  df.drop('distance', axis=1, inplace=True)
  return df

### Discussion

- The cognitive model that we have described above accurately predicts human gaze-based interaction performance (Chen et al., 2021).
- It is an example of a **computationally rational** cognitive model. This is because the behaviour is predicted from an approximately optimal policy given hypothesised bounds on cognition.
- It should be possible to find an approximately optimal policy using any reinforcement learning algorithm.  The only difference that the algorithm will make is to the efficiency with which solution is found.
- The separation of cognitive theory and reinforcement learning algorithm is achieved through the statement of the architecture as what is known as a belief-state Markov Decision Process (a **belief-state MDP**), which is a type of Partially Observable Markov Decision Process (POMDP).
- Using reinforcement learning (RL) to model cognition with the approximately optimal policy contrast does not model the human learning process. For work that does use RL to model human learning see Daw and Dayan (2008).
- A number of CHI papers have made use of this architecture. See Oulasvirta et al., 2022 for a review.
- An important issue concerns how model parameters are fitted to data. See Keuralinen et al., 2023.

### References
Chen, X., Acharya, A., Oulasvirta, A., & Howes, A. (2021, May). An adaptive model of gaze-based selection. In Proceedings of the 2021 CHI Conference on Human Factors in Computing Systems (pp. 1-11).

Chen, H., Chang, H. J., & Howes, A. (2021, May). Apparently Irrational Choice as Optimal Sequential Decision Making. In Proceedings of the AAAI Conference on Artificial Intelligence (Vol. 35, No. 1, pp. 792-800).

Dayan, P., & Daw, N. D. (2008). Decision theory, reinforcement learning, and the brain. Cognitive, Affective, & Behavioral Neuroscience, 8(4), 429-453.

Oulasvirta, A., Jokinen, J. P., & Howes, A. (2022, April). Computational rationality as a theory of interaction. In Proceedings of the 2022 CHI Conference on Human Factors in Computing Systems (pp. 1-14).

Keurulainen, A., Westerlund, I. R., Keurulainen, O., & Howes, A. (2023, April). Amortised Experimental Design and Parameter Estimation for User Models of Pointing. In Proceedings of the 2023 CHI Conference on Human Factors in Computing Systems (pp. 1-17).